# Video Detection Demo with PytorchWildlife

This tutorial demonstrates the capabilities of the PytorchWildlife package for video detection and classification. We will go through the process of setting up the environment, applying detection and classification on video frames, and saving the results in an annotated video format.

## Prerequisites
Ensure that the PytorchWildlife package and other required libraries are installed. Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU.

## Setup
First, let's import the necessary libraries and modules.

In [ ]:
from PIL import Image
import numpy as np
import supervision as sv
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife import utils as pw_utils

## Setting GPU
Specify which GPU to use for the computations. By default, GPU number 0 is used. Adjust this as per your setup.

In [ ]:
torch.cuda.set_device(0) # Use only if you are running on GPU

## Model Initialization
We'll initialize models for both video detection and classification.

In [ ]:
DEVICE = "cpu" # Use CUDA if you are running on GPU
detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True)
classification_model = pw_classification.AI4GOpossum(device=DEVICE, pretrained=True)

## Transformations
Define transformations for both detection and classification. These transformations preprocess the video frames for the models.

In [ ]:
trans_det = pw_trans.MegaDetector_v5_Transform(target_size=detection_model.IMAGE_SIZE,
                                               stride=detection_model.STRIDE)
trans_clf = pw_trans.Classification_Inference_Transform(target_size=224)

## Video Processing
For each frame in the video, we'll apply detection and classification, and then annotate the frame with the results. The processed video will be saved with annotated detections and classifications.

In [ ]:
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results_det = detection_model.single_image_detection(trans_det(frame), frame.shape, index)
    labels = []
    for xyxy in results_det["detections"].xyxy:
        cropped_image = sv.crop_image(image=frame, xyxy=xyxy)
        results_clf = classification_model.single_image_classification(trans_clf(Image.fromarray(cropped_image)))
        labels.append("{} {:.2f}".format(results_clf["prediction"], results_clf["confidence"]))
    annotated_frame = box_annotator.annotate(scene=frame, detections=results_det["detections"], labels=labels)
    return annotated_frame 

pw_utils.process_video(source_path=SOURCE_VIDEO_PATH, target_path=TARGET_VIDEO_PATH, callback=callback, target_fps=5)

## Conclusion
This tutorial covered video detection and classification using the PytorchWildlife package. You can further adapt and expand this demo according to your requirements.